Prediksi Harga Crude Palm Oil (CPO) di Indonesia dengan Menggunakan Metode Fuzzy Time Series Markov Chain

In [4]:
import numpy as np
import pandas as pd

class FuzzyTimeSeriesMarkovChain:
    def __init__(self):
        self.universe = []
        self.intervals = []
        self.midpoints = []
        self.fuzzy_sets = []
        self.fuzzified = []
        self.flr = []
        self.flrg = {}
        self.transition_matrix = None
        self.min_val = None
        self.max_val = None
        self.k = None
        self.interval_length = None

    def generate_universe(self, data):
        """Menentukan himpunan semesta U"""
        self.min_val = min(data)
        self.max_val = max(data)
        self.universe = [self.min_val, self.max_val]

    def generate_intervals(self, data):
        """Menentukan interval dan nilai tengah"""
        n = len(data)
        # Menentukan jumlah interval dengan Sturges Rule
        k = int(1 + 3.322 * np.log10(n))

        # Menentukan panjang interval
        interval_length = (self.max_val - self.min_val) / k

        # Membuat intervals
        intervals = []
        midpoints = []
        for i in range(k):
            lower = self.min_val + (i * interval_length)
            upper = self.min_val + ((i+1) * interval_length)
            intervals.append([lower, upper])
            midpoints.append((lower + upper) / 2)

        self.intervals = intervals
        self.midpoints = midpoints
        self.k = k
        self.interval_length = interval_length

    def fuzzify(self, data):
        """Melakukan fuzzifikasi data"""
        fuzzified = []
        for value in data:
            for i, interval in enumerate(self.intervals):
                if interval[0] <= value <= interval[1]:
                    fuzzified.append(f"A{i+1}")
                    break
        self.fuzzified = fuzzified

    def generate_flr(self):
        """Menentukan Fuzzy Logic Relationship"""
        for i in range(len(self.fuzzified)-1):
            self.flr.append([self.fuzzified[i], self.fuzzified[i+1]])

    def generate_flrg(self):
        """Menentukan Fuzzy Logic Relationship Group"""
        for current, next_state in self.flr:
            if current not in self.flrg:
                self.flrg[current] = []
            self.flrg[current].append(next_state)

    def generate_transition_matrix(self):
        """Menentukan matriks probabilitas transisi"""
        matrix_size = len(self.intervals)
        self.transition_matrix = np.zeros((matrix_size, matrix_size))

        for current in self.flrg:
            current_idx = int(current.replace('A', '')) - 1
            next_states = self.flrg[current]

            for next_state in next_states:
                next_idx = int(next_state.replace('A', '')) - 1
                self.transition_matrix[current_idx][next_idx] += 1

        # Normalisasi matrix
        row_sums = self.transition_matrix.sum(axis=1)
        for i in range(len(self.transition_matrix)):
            if row_sums[i] != 0:
                self.transition_matrix[i] = self.transition_matrix[i] / row_sums[i]

    def forecast(self, data):
        """Melakukan peramalan"""
        forecasted = []

        for i in range(len(data)):
            if i == 0:
                forecasted.append(data[0])
                continue

            current_fuzzy = self.fuzzified[i-1]
            current_idx = int(current_fuzzy.replace('A', '')) - 1

            # Menghitung nilai peramalan berdasarkan matriks probabilitas
            forecast_value = 0
            for j in range(len(self.intervals)):
                prob = self.transition_matrix[current_idx][j]
                if prob > 0:
                    forecast_value += self.midpoints[j] * prob

            # Menambahkan nilai penyesuaian
            if i > 1:
                prev_idx = int(self.fuzzified[i-2].replace('A', '')) - 1
                curr_idx = int(self.fuzzified[i-1].replace('A', '')) - 1

                if prev_idx < curr_idx:  # Trend naik
                    adjustment = self.interval_length / 2
                elif prev_idx > curr_idx:  # Trend turun
                    adjustment = -self.interval_length / 2
                else:
                    adjustment = 0

                forecast_value += adjustment

            forecasted.append(forecast_value)

        return forecasted

    def calculate_mape(self, actual, predicted):
        """Menghitung Mean Absolute Percentage Error"""
        actual = np.array(actual)
        predicted = np.array(predicted)
        return np.mean(np.abs((actual - predicted) / actual)) * 100

    def fit_predict(self, data):
        """Melakukan seluruh proses prediksi"""
        self.generate_universe(data)
        self.generate_intervals(data)
        self.fuzzify(data)
        self.generate_flr()
        self.generate_flrg()
        self.generate_transition_matrix()
        return self.forecast(data)

# Contoh penggunaan
if __name__ == "__main__":
    # Data contoh (gunakan data aktual dari jurnal)
    data = [
        # Masukkan data harga CPO dari Januari 2019 - Oktober 2022
        5924, 6000, 6200, 6500, 6800, 7000  # Contoh beberapa data
    ]

    # Inisialisasi model
    model = FuzzyTimeSeriesMarkovChain()

    # Melakukan prediksi
    predictions = model.fit_predict(data)

    # Menghitung MAPE
    mape = model.calculate_mape(data, predictions)
    accuracy = 100 - mape

    print(f"MAPE: {mape:.6f}%")
    print(f"Accuracy: {accuracy:.6f}%")

    # Prediksi untuk bulan berikutnya
    last_month = predictions[-1]
    print(f"Prediksi bulan berikutnya: Rp {last_month:.2f}/kg")


MAPE: 1.881402%
Accuracy: 98.118598%
Prediksi bulan berikutnya: Rp 7000.00/kg


Kode ini mengimplementasikan model Fuzzy Time Series berbasis Rantai Markov untuk melakukan peramalan data deret waktu. Berikut poin-poin utama dari model ini:
1.	Tahapan Utama:

•	Himpunan Semesta (Universe): Menentukan rentang nilai dari data input.

•	Pembentukan Interval: Membagi data menjadi beberapa interval berdasarkan aturan Sturges, lalu menghitung nilai tengahnya.

•	Fuzzifikasi: Mengonversi data numerik menjadi simbol fuzzy (A1, A2, dst.) berdasarkan interval yang terbentuk.

•	Hubungan Logika Fuzzy (FLR dan FLRG): Menentukan hubungan logika antar simbol fuzzy dan mengelompokkannya.

•	Matriks Transisi: Membentuk matriks probabilitas untuk memodelkan transisi antar simbol fuzzy.

•	Peramalan: Menggunakan matriks transisi dan nilai tengah untuk memprediksi data berikutnya.

2.	Keunggulan Model:
•	Menggabungkan konsep Fuzzy Logic (untuk menangani ketidakpastian) dengan Markov Chain (untuk model transisi probabilistik).

•	Cocok untuk data deret waktu dengan pola yang tidak pasti dan fluktuatif.

3.	Akurasi Model:
•	Model menghitung kesalahan menggunakan MAPE (Mean Absolute Percentage Error), yang memberikan gambaran tingkat keakuratan peramalan.

4.	Penggunaan:
•	Skrip ini dapat digunakan untuk peramalan data waktu nyata, misalnya harga komoditas (seperti harga CPO dalam contoh).

5.	Hasil Akhir:
•	Memberikan prediksi nilai masa depan berdasarkan data historis.
•	Menampilkan tingkat kesalahan (MAPE) dan akurasi prediksi.

